# Importation des packages 

Nous chargons les modules necessaires pour notre analyse. 

In [16]:
#Importation des packages

import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from kmodes.kmodes import KModes
import os 

In [17]:
# Definir le repertoire de travail 
pwd = "C:\\Users\\mouss\\OneDrive\\Documents\\TotalEnergies" 
os.chdir(pwd) 
 
# Afficher le nouveau repertoire de travail 
print(os.getcwd()) 

C:\Users\mouss\OneDrive\Documents\TotalEnergies


# Données
Nous chargeons le jeu de données de la correspondance supports-clients. Cette table contient 291 964 lignes et 6 facteurs. Tous les facteurs sont des variables à modalités sauf le code des clients. 

In [18]:
# Importation des données

data = pd.read_csv("mapping_support.csv",sep = ";"
                       #,index_col = "EP_CODE"
                      , encoding= 'latin-1')

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291964 entries, 0 to 291963
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   EP_CODE            291964 non-null  int64 
 1   SUPPORT_CODE       291962 non-null  object
 2   SUPPORT_TYPE       291962 non-null  object
 3   SUPPORT_STATUS     291964 non-null  object
 4   SUPPORT_SUBSTATUS  291964 non-null  object
 5   VEHICLE_TYPE       291961 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.4+ MB


# Echantillon
Ci dessous un échantilon de cette table.

In [20]:
data.head()

,EP_CODE,SUPPORT_CODE,SUPPORT_TYPE,SUPPORT_STATUS,SUPPORT_SUBSTATUS,VEHICLE_TYPE
0,87049492,C700,Card,Actif,En activité,04-OTHER
1,87289912,C700,Card,Actif,En activité,03-PASSENGER VEHICLE
2,87274364,C700,Card,Actif,En activité,03-PASSENGER VEHICLE
3,87267522,C700,Card,Actif,En activité,03-PASSENGER VEHICLE
4,87310757,C700,Card,Actif,En activité,01-HEAVY GOODS VEHICLE


# Nettoyage et preparation de données

## Types des variables

Dans cette partie, il s'agit de regarder de plus près le type de chaque variable et decider s'il est pertinent de les convertir vers d'autres types. Pour ce faire, analysons la sortie suivante : 

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291964 entries, 0 to 291963
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   EP_CODE            291964 non-null  int64 
 1   SUPPORT_CODE       291962 non-null  object
 2   SUPPORT_TYPE       291962 non-null  object
 3   SUPPORT_STATUS     291964 non-null  object
 4   SUPPORT_SUBSTATUS  291964 non-null  object
 5   VEHICLE_TYPE       291961 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.4+ MB


On remaruqe que toute les variables sont de type *object* (correspondant à string) sauf EP_CODE qui est de type entier. Le type *object* ne pose aucun problème pour les variables qui sont de nature categorielles. C'est le cas pour toutes les variables dans cette table sauf pour EP_CODE bien sur. 

## Nombre de modalités par variables

In [22]:
# Connaitre le nombre de valeurs uniques pour toutes les colonnes dans le but de savoir quelles variables convertir en categorielle

unique_counts = pd.DataFrame.from_records([(col, data[col].nunique()) for col in data.columns],
                          columns=['Nom_variables', 'Nombre_valeurs_uniques']).sort_values(by=['Nombre_valeurs_uniques'])

unique_counts

,Nom_variables,Nombre_valeurs_uniques
3,SUPPORT_STATUS,1
4,SUPPORT_SUBSTATUS,1
2,SUPPORT_TYPE,3
5,VEHICLE_TYPE,4
1,SUPPORT_CODE,40
0,EP_CODE,151137


## Données manquantes

Dans cette table, il y'a très peu de données manquantes. La variable qui a le plus de données manquantes est *VEHICULE_TYPE* avec seulement 3 NA. En ignorant les lignes contenant une données manquantes de cette table, on enlèvera au maximum 7 lignes. Ce qui est negligeable et aura très peu d'effets sur les analyses. Nous allons donc supprimer les lignes comportant des données manquantes (case vide).

In [23]:
# Données manquantes

# Connaitre le nombre de valeurs uniques pour toutes les colonnes dans le but de savoir quelles variables convertir en categorielle

NA_counts = pd.DataFrame.from_records([(col
                                        , data[col].isna().sum()
                                       , round((data[col].isna().sum())/(data[col].size),3)) for col in data.columns],
                          columns=['Nom_variables', 'Nombre_valeurs_manquantes',"%NA"]).sort_values(by=['Nombre_valeurs_manquantes'], ascending=False)

NA_counts

,Nom_variables,Nombre_valeurs_manquantes,%NA
5,VEHICLE_TYPE,3,0.0
1,SUPPORT_CODE,2,0.0
2,SUPPORT_TYPE,2,0.0
0,EP_CODE,0,0.0
3,SUPPORT_STATUS,0,0.0
4,SUPPORT_SUBSTATUS,0,0.0


In [24]:
data = data.copy().dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 291961 entries, 0 to 291963
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   EP_CODE            291961 non-null  int64 
 1   SUPPORT_CODE       291961 non-null  object
 2   SUPPORT_TYPE       291961 non-null  object
 3   SUPPORT_STATUS     291961 non-null  object
 4   SUPPORT_SUBSTATUS  291961 non-null  object
 5   VEHICLE_TYPE       291961 non-null  object
dtypes: int64(1), object(5)
memory usage: 15.6+ MB


# Description sommaire des données de la table de correspondance client-suppot

Le tableau ci dessous decrit brievement chaque variable. Il y'a dans ce tableau pour chque variable :
* count : le nombre d'observations (exclu les données manquantes :NAN)
* unique : le nombre de valeurs (modalités) discincts
* top : la valeur la plus fréquente
* freq : la frequence de la valeur la plus frequente (nombre brute=> comptage)
* freq_p : la proportion de la valeur la plus frequente ( en %)

Ainsi, on peut remarquer que parmi les 3 types de support, le type **card** (>79%) est le plus utilisé par les clients. Il faut noter aussi que tous les supports dans cette table ont le statut **actif**, donc sont en service.

In [25]:
stats = data.iloc[:,1:].describe()  # Ne pas considerer la variable EP_CODE.
#stats

In [26]:
# Ajout de la frequence en %

stats.loc["freq_p"] = pd.Series(stats.loc["freq"]/stats.loc["count"]).apply(lambda value : round(value,3)) # frequence en pourcentage
stats

,SUPPORT_CODE,SUPPORT_TYPE,SUPPORT_STATUS,SUPPORT_SUBSTATUS,VEHICLE_TYPE
count,291961,291961,291961,291961,291961
unique,40,3,1,1,4
top,C700,Card,Actif,En activité,04-OTHER
freq,116312,230880,291961,291961,98796
freq_p,0.398,0.791,1.0,1.0,0.338


# Autres analyses
Les types de vehicules le plus utilisés par les clients (compositon de la flotte) est **04-OTHER** (33%) secondé par **03-PASSENGER VEHICLE** (20%) et le moins utilisé est de type **HEAVY GOODS VEHICLE** (7,1%).

In [27]:
data.value_counts("VEHICLE_TYPE",normalize=True)

VEHICLE_TYPE
04-OTHER                                  0.338388
03-PASSENGER VEHICLE                      0.304219
02-LIGHT COMMERCIAL/COMMERCIAL VEHICLE    0.285511
01-HEAVY GOODS VEHICLE                    0.071883
dtype: float64

In [28]:
data.value_counts("SUPPORT_TYPE",normalize=True)

SUPPORT_TYPE
Card                 0.790791
Badge                0.208110
Boitier WayKonect    0.001099
dtype: float64

# Enregistrer les données propres

In [29]:
data.to_csv('./data/mapping_support.csv',sep=';',encoding='latin-1',index=False)